# Modelos en Django

Lo que queremos crear ahora es algo que almacene todas las entradas de nuestro blog. Pero para poder hacerlo tenemos que hablar un poco sobre algo llamado `objetos`.

# Objetos

Hay un concepto en el mundo de la programación llamado `programación orientada a objetos`. La idea es que en lugar de escribir todo como una aburrida secuencia de instrucciones de programación podemos modelar cosas y definir cómo interactúan entre ellas.

Entonces, ¿qué es un objeto? Es un conjunto de propiedades y acciones. Suena raro, pero te daremos un ejemplo.

Si queremos modelar un gato crearemos un objeto `Gato` que tiene algunas propiedades como: `color`, `edad`, `temperamento` (como bueno, malo, o dormilón ;)), y `dueño` (este es un objeto Persona o en caso de un gato callejero, esta propiedad está vacía).

Luego, el `Gato` tiene algunas acciones como: `ronronear`, `arañar` o `alimentar` (en cuyo caso daremos al gato algo de `ComidaDeGato`, el cual debería ser un objeto aparte con propiedades como `sabor`).

```
Gato
---------
color 
edad 
humor 
dueño 
ronronear() 
rasguñar() 
alimentarse(comida_de_gato) 

ComidaDeGato
----------
sabor

```

Básicamente se trata de describir cosas reales en el código con propiedades (llamadas `propiedades del objeto`) y las acciones (llamadas `métodos`).

Y ahora, ¿cómo modelamos las entradas en el blog? Queremos construir un blog, ¿no?

Necesitamos responder a la pregunta: ¿Qué es una entrada de un blog? ¿Qué propiedades debería tener?

Bueno, seguro que nuestras entradas de blog necesitan un texto con su contenido y un título, ¿cierto? También sería bueno saber quién lo escribió, así que necesitamos un autor. Por último, queremos saber cuándo se creó y publicó la entrada.

```
Post
--------
title
text
author
created_date
published_date
```

¿Qué tipo de cosas podría hacerse con una entrada del blog? Sería bueno tener algún `método` que publique la entrada, ¿no?

Así que vamos a necesitar el método `publicar`.

Puesto que ya sabemos lo que queremos lograr, ¡podemos empezar a modelarlo en Django!

# Modelos en Django

Sabiendo qué es un objeto, podemos crear un modelo en Django para nuestros entradas de blog.

Un modelo en Django es un tipo especial de objeto que se guarda en la `base de datos`. Una base de datos es una colección de datos. Es un lugar en el cual almacenarás la información sobre usuarios, tus entradas de blog, etc. Utilizaremos una base de datos SQLite para almacenar nuestros datos. Este es el adaptador de base de datos predeterminado en Django -- será suficiente para nosotros por ahora.

Puedes pensar el modelo en la base de datos, como una hoja de cálculo con columnas (campos) y filas (datos).


# Crear una aplicación

Para mantener todo en orden, crearemos una aplicación separada dentro de nuestro proyecto. Es muy bueno tener todo organizado desde el principio. Para crear una aplicación, necesitamos ejecutar el siguiente comando en la consola (dentro de la carpeta de blog donde está el archivo manage.py):

```
(myvenv) ~/blog$ python manage.py startapp blog_curso
```

Notarás que se ha creado un nuevo directorio blog_curso y ahora contiene una cantidad de archivos. Los directorios y archivos en nuestro proyecto deberían verse así:

```
blog
├── blog_curso
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
├── db.sqlite3
├── manage.py
├── mysite
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── requirements.txt
```

Después de crear una aplicación, también necesitamos decirle a Django que debe utilizarla. Eso se hace en el fichero blog/settings.py -- ábrelo en el editor. Tenemos que encontrar INSTALLED_APPS y agregar una línea que contiene 'blog', justo por encima de ]. El producto final debe tener este aspecto:

```
# blog/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',

    'blog_curso',
]
```

# Crear el modelo del Post

```python
# blog/blog/models.py
from django.db import models
from django.utils import timezone


class Post(models.Model):
    author = models.ForeignKey('auth.User', on_delete=models.CASCADE)
    title = models.CharField(max_length=200)
    text = models.TextField()
    created_date = models.DateTimeField(
            default=timezone.now, blank=False, null=False)
    published_date = models.DateTimeField(
            blank=True, null=True)

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title
```

Todas las líneas que comienzan con `from` o `import` son líneas para agregar algo de otros archivos. Así que en vez de copiar y pegar las mismas cosas en cada archivo, podemos incluir algunas partes con `from... import ....`


`class Post(models.Model):` esta línea define nuestro modelo (es un `objeto`).

 * `class` es una palabra clave que indica que estamos definiendo un objeto.
 * `Post` es el nombre de nuestro modelo. Podemos darle un nombre diferente (pero debemos evitar espacios en blanco y caracteres especiales). Siempre inicia el nombre de una clase con una letra mayúscula.
 * `models.Model` significa que Post es un modelo de Django, así Django sabe que debe guardarlo en la base de datos.


Ahora definimos las propiedades de las que hablábamos: `title`, `text`, `created_date`, `published_date` y `author`. Para ello tenemos que definir el tipo de cada campo (¿es texto? ¿un número? ¿una fecha? ¿una relación con otro objeto como un User (usuario)?)

 * `models.CharField`, así es como defines un texto con un número limitado de caracteres.
 * models.TextField, este es para texto largo sin límite. Suena perfecto para el contenido de la entrada del blog, ¿no?
 * `models.DateTimeField`, este es fecha y hora.
 * `modelos.ForeignKey`, este es una relación (link) con otro modelo.


# Crear tablas para los modelos en tu base de datos

El último paso aquí es agregar nuestro nuevo modelo a la base de datos. Primero tenemos que hacer saber a Django que hemos hecho cambios en nuestro modelo. (Lo acabamos de crear!) Ve a tu terminal y escribe python manage.py makemigrations blog. Se verá así:

```
(myvenv) ~/blog$ python manage.py makemigrations blog_curso
Migrations for 'blog_curso':
  blog_curso/migrations/0001_initial.py:

  - Create model Post
```

Django preparó un archivo de migración que ahora tenemos que aplicar a nuestra base de datos. Escribe `python manage.py migrate blog_curso` y el resultado debería ser:

```
(myvenv) ~/blog$ python manage.py migrate blog_curso
Operations to perform:
  Apply all migrations: blog
Running migrations:
  Applying blog.0001_initial... OK
```

```
$ python manage.py migrate # migraciones de todo el proyecto
$ python manage.py createsuperuser
Username (leave blank to use 'user'): admin
Email address: admin@admin.com
Password:
Password (again):
The password is too similar to the username.
This password is too short. It must contain at least 8 characters.
This password is too common.
Bypass password validation and create user anyway? [y/N]: y
Superuser created successfully.
```

# Correr el proyecto django

```
$ python manage.py runserver
```

```
from django.contrib import admin

from .models import Post

admin.site.register(Post)
```

# Modificar views.py

```python
from django.views.generic import ListView

from .models import Post


class PostListView(ListView):

    model = Post
    template_name = "blog_curso/post_list.html"


posts = PostListView.as_view()
```